# Introduction

## Kubectl client version

Firs of all let's take a look at the kubectl client version.


In [34]:
!kubectl version --client -o json 

{
  "clientVersion": {
    "major": "1",
    "minor": "24",
    "gitVersion": "v1.24.1",
    "gitCommit": "3ddd0f45aa91e2f30c70734b175631bec5b5825a",
    "gitTreeState": "clean",
    "buildDate": "2022-05-24T12:26:19Z",
    "goVersion": "go1.18.2",
    "compiler": "gc",
    "platform": "linux/amd64"
  },
  "kustomizeVersion": "v4.5.4"
}


## Azure client

Firsr of all, let's take a look at the Azure client version.


In [35]:
!az version

{
  "azure-cli": "2.37.0",
  "azure-cli-core": "2.37.0",
  "azure-cli-telemetry": "1.0.6",
  "extensions": {}
}


### Before you start

Please, make sure to have setup a [Kubernetes cluster on Microsoft Azure](https://github.com/GeoLabs/ADES-Azure-Introduction-tale/blob/main/BeforeYouStart.md) and use the correct reference to it.




### Login

When you first load this page, you are invited to login to the Azure cluster.

In [ ]:
!az login

In [37]:
!az account set --subscription VVVVVVVV-WWWW-XXXX-YYYY-ZZZZZZZZZZZZ

In [148]:
!rm -rf ~/.kube ; az aks get-credentials --resource-group ades-azure-demo-001_group --name ades-azure-demo-001

Merged "ades-azure-demo-001" as current context in /home/jovyan/.kube/config


In [100]:
!kubectl get deployments --all-namespaces=true

NAMESPACE     NAME                 READY   UP-TO-DATE   AVAILABLE   AGE
kube-system   aks-link             2/2     2            2           35m
kube-system   coredns              2/2     2            2           35m
kube-system   coredns-autoscaler   1/1     1            1           35m
kube-system   metrics-server       1/1     1            1           35m
kube-system   omsagent-rs          1/1     1            1           35m


In [115]:
!kubectl get deployments --all-namespaces=true

NAMESPACE     NAME                 READY   UP-TO-DATE   AVAILABLE   AGE
kube-system   aks-link             2/2     2            2           8m1s
kube-system   coredns              5/5     5            5           8m5s
kube-system   coredns-autoscaler   1/1     1            1           8m5s
kube-system   metrics-server       1/1     1            1           8m5s
kube-system   omsagent-rs          1/1     1            1           8m5s


In [77]:
!kubectl get pods --all-namespaces=true

NAMESPACE     NAME                                 READY   STATUS    RESTARTS   AGE
kube-system   azure-ip-masq-agent-jhxpc            1/1     Running   0          9m21s
kube-system   azure-ip-masq-agent-nhbt9            1/1     Running   0          9m32s
kube-system   azure-ip-masq-agent-pnb6d            1/1     Running   0          8m30s
kube-system   cloud-node-manager-5ssl6             1/1     Running   0          8m30s
kube-system   cloud-node-manager-c87g7             1/1     Running   0          9m21s
kube-system   cloud-node-manager-xncpw             1/1     Running   0          9m32s
kube-system   coredns-autoscaler-7d56cd888-c6xq6   1/1     Running   0          10m
kube-system   coredns-dc97c5f55-7v5rt              1/1     Running   0          8m41s
kube-system   coredns-dc97c5f55-zp8rd              1/1     Running   0          10m
kube-system   csi-azuredisk-node-qsvtq             3/3     Running   0          9m21s
kube-system   csi-azuredisk-node-s5rfp             3/3     R

# MiniIO



In [117]:
!kubectl create ns minio

namespace/minio created


In [118]:
!kubectl create -f k8s/my-minio-fs.yaml -n minio

persistentvolumeclaim/my-minio-fs-pvc created
deployment.apps/my-minio-fs created


In [120]:
!kubectl expose deployment/my-minio-fs --type="NodePort" --port 9000 -n minio

service/my-minio-fs exposed


In [119]:
!echo "run sh k8s/startMinio.sh from a terminal"

run sh k8s/startMinio.sh from a terminal


In [121]:
!curl http://127.0.0.1:9000/minio -v

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to 127.0.0.1 (127.0.0.1) port 9000 (#0)
> GET /minio HTTP/1.1
> Host: 127.0.0.1:9000
> User-Agent: curl/7.58.0
> Accept: */*
> 
< HTTP/1.1 403 Forbidden
< Accept-Ranges: bytes
< Content-Type: application/xml
< Server: Minio/RELEASE.2017-11-22T19-55-46Z (linux; amd64)
< Vary: Origin
< X-Amz-Request-Id: 16F64E15EC62EFFE
< Date: Tue, 07 Jun 2022 09:56:09 GMT
< Transfer-Encoding: chunked
< 
<?xml version="1.0" encoding="UTF-8"?>
* Connection #0 to host 127.0.0.1 left intact
<Error><Code>AllAccessDisabled</Code><Message>All access to this bucket has been disabled.</Message><Key></Key><BucketName></BucketName><Resource>/minio</Resource><RequestId>3L137</RequestId><HostId>3L137</HostId></Error>

# Install ADES on Kubernetes cluster

This section guides you through the deploy of the Ades on a Kubernetes cluster.

## Create a dedicated namespace for the ADES

This namespace will be reserved for the operation of the ADES in order to separate the ADES proper execution from its processings job on kubernetes.

For instance, let's create namespace `eoepca`


In [122]:
!kubectl create namespace eoepca

namespace/eoepca created


## Deploy using Helm

The ADES is "installed" via helm specifying an optional "release name".

for instance, we will install the ADES with the name `ades`.


In [123]:
!cd ; rm -rf proc-ades ; \
    git clone https://github.com/EOEPCA/proc-ades.git ; \
    cp ${REPO_DIR}/k8s/values.testbed18.geolabs.yaml proc-ades/charts/ades/values.yaml

Cloning into 'proc-ades'...
remote: Enumerating objects: 8535, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 8535 (delta 61), reused 53 (delta 47), pack-reused 8451
Receiving objects: 100% (8535/8535), 12.88 MiB | 22.39 MiB/s, done.
Resolving deltas: 100% (4485/4485), done.


In [124]:
!cd ; cd proc-ades ; helm install ades charts/ades/ --namespace eoepca

W0607 09:56:30.208588    5014 warnings.go:70] policy/v1beta1 PodSecurityPolicy is deprecated in v1.21+, unavailable in v1.25+
W0607 09:56:35.968684    5014 warnings.go:70] policy/v1beta1 PodSecurityPolicy is deprecated in v1.21+, unavailable in v1.25+
NAME: ades
LAST DEPLOYED: Tue Jun  7 09:56:29 2022
NAMESPACE: eoepca
STATUS: deployed
REVISION: 1
NOTES:
1. Get the application URL by running these commands:
  export POD_NAME=$(kubectl get pods --namespace eoepca -l "app.kubernetes.io/name=ades,app.kubernetes.io/instance=ades" -o jsonpath="{.items[0].metadata.name}")
  echo "Visit http://127.0.0.1:8080 to use your application"
  kubectl --namespace eoepca port-forward $POD_NAME 8080:80


In [147]:
!kubectl get pods --namespace eoepca

NAME                              READY   STATUS    RESTARTS   AGE
ades-69cb587757-pxk7j             1/1     Running   0          30m
ades-kubeproxy-84b47f7f66-pwcvp   1/1     Running   0          30m


In [128]:
!kubectl get service ades --namespace eoepca

NAME   TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)   AGE
ades   ClusterIP   10.0.249.33   <none>        80/TCP    61s


## Access the application endpoint

Get the application URL by running these commands:



In [132]:
!echo 'run the following command: export POD_NAME=$(kubectl get pods --namespace eoepca -l "app.kubernetes.io/name=ades,app.kubernetes.io/instance=ades" -o jsonpath="{.items[0].metadata.name}") ; kubectl --namespace eoepca port-forward $POD_NAME 8080:80 from a terminal'

run the following command: export POD_NAME=$(kubectl get pods --namespace eoepca -l "app.kubernetes.io/name=ades,app.kubernetes.io/instance=ades" -o jsonpath="{.items[0].metadata.name}") ; kubectl --namespace eoepca port-forward $POD_NAME 8080:80 from a terminal


The Ades is now deployed on your Kubernetes cluster and is ready to use. The WPS endpoint is reachable on the port 8080, to see how to make a request please proceed to the section [ADES/Introduction](ADES/Introduction.ipynb).